Benchmark Queries per second(QPS) or Transaction per second(TPS)

v1.6: 15 May
* Added option to filter models list to run selected models behcmark and to save time

v1.5: 14 May  2024
* Modified code to run Keras cv attention models first than tf keras. So if any issue in that it will be detected early.
* reduced batch size and number of iteration to save time.
* Skipped detection models from benchmark

v1.0: 14 May  2024

* Added support to check model QPS (Queries per second) or we can say Transactions per second
* Used tf dataset
* tested on v2.15.0  keras, tf_keras and tensorflow

## <font color="#000080" size="+3">1. Setup</font>


In [1]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-decision-forests, tensorflow-serving-api, tensorflow-text, tf_keras, witwidget


In [2]:
!pip install tf_keras=="2.15.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: tf_keras
    Found existing installation: tf_keras 2.15.1
    Uninstalling tf_keras-2.15.1:
      Successfully uninstalled tf_keras-2.15.1


In [3]:
# !pip install keras_cv_attention_models
!pip install keras-cv-attention-models==1.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.3/796.3 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [4]:
# set environment variable values before importing tensorflow and keras
import os

os.environ["KECAM_BACKEND"] = "tensorflow"
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [5]:
!pip install nb_utils

In [6]:
# !pip  uninstall -y tf_tools
!pip install git+https://github.com/Nivratti/tf_tools.git#egg=tf_tools

  Cloning https://github.com/Nivratti/tf_tools.git to /tmp/pip-install-lnuwrkpz/tf-tools_627bc976c94e4b73aa335587c6ef74dc
  Running command git clone --filter=blob:none --quiet https://github.com/Nivratti/tf_tools.git /tmp/pip-install-lnuwrkpz/tf-tools_627bc976c94e4b73aa335587c6ef74dc
  Resolved https://github.com/Nivratti/tf_tools.git to commit 9b53934b0f7194d192b3b6c8b412c0f24fa1a3a2
  Preparing metadata (setup.py) ... done
  Created wheel for tf_tools: filename=tf_tools-1.1.0-py3-none-any.whl size=27074 sha256=49b348b6c89bc53b672a0bb5f25c3859f6861b288d3bed5c8615ad05295e9fa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-spov0l4r/wheels/47/2f/43/d3226af5507f8cb43d9bd1655817b7954246fb1929d95953d7
Successfully built tf_tools


## <font color="#000080" size="+3">2. Import libraries</font>


In [7]:
import numpy as np
import time
import tf_keras
from tf_keras.applications import *
from tf_keras.preprocessing import image
from tf_keras.utils import get_file
import tensorflow as tf
import pandas as pd
import inspect 
from tqdm import tqdm

2024-05-15 07:16:07.425630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 07:16:07.425754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 07:16:07.536665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## <font color="#000080" size="+3">3. Useful functions</font>


In [8]:
## useful functions

def find_max_batch_size(model, input_size):
    """
    Finds the maximum batch size that can fit into GPU memory without causing a ResourceExhaustedError.

    Args:
        model (tf.keras.Model): The model for which to find the maximum batch size.
        input_size (int): The height and width of the input images. Assumes square images.

    Returns:
        int: The maximum batch size that can be processed by the model without memory overflow.
    """
    batch_size = 2
    max_batch_size = 0

    while True:
        try:
            # Create dummy data
            data = np.random.rand(batch_size, input_size, input_size, 3).astype('float32')
            # Predict
            preds = model.predict(data)
            max_batch_size = batch_size
            batch_size *= 2  # Double the batch size
        except tf.errors.ResourceExhaustedError:
            break  # Out of memory
        finally:
            tf_keras.backend.clear_session()
    return max_batch_size

def benchmark_model(
        model, strategy, input_shape, 
        warm_up_batches=10, num_batches=100, 
        batch_size=32
    ):
    """
    Benchmarks a TensorFlow model to estimate its queries per second (QPS) under a specific distribution strategy.

    Args:
        model (tf.keras.Model): The model to benchmark.
        strategy (tf.distribute.Strategy): The distribution strategy under which the benchmark is performed.
        input_shape (tuple): The shape of the input data (excluding the batch size).
        warm_up_batches (int, optional): Number of warm-up batches to run before timing to stabilize performance metrics.
        num_batches (int, optional): Number of batches to process for benchmarking.
        batch_size (int, optional): Number of samples per batch.

    Returns:
        tuple: A tuple containing the estimated queries per second (QPS) and the total time taken for the benchmark (in seconds).
    """
    input_data = np.random.random((batch_size, *input_shape)).astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices(input_data).repeat().batch(batch_size)

    # Warm-up
    with strategy.scope():
        print(f"performing warmup...")
        for images in dataset.take(warm_up_batches):
            # _ = model.predict_on_batch(images)
            _ = model(images)  # To make it work for keras-cv_attention_models

    start_time = time.time()
    with strategy.scope():
        print(f"Benchmarking...")
        for batch in dataset.take(num_batches):
            _ = model(batch)
            
    total_time = time.time() - start_time

    total_queries = num_batches * batch_size
    qps = total_queries / total_time
    return qps, total_time

## <font color="#000080" size="+3">4. Device statergy</font>


In [9]:
from tf_tools.device_strategy import get_device_strategy
# specify only one core
strategy, device_type = get_device_strategy(num_cores_to_use=1)

2024-05-15 07:16:17.695 | INFO     | tf_tools.device_strategy:get_device_strategy:41 - Multiple GPUs are available, using MirroredStrategy. Count: 2
2024-05-15 07:16:18.213 | INFO     | tf_tools.device_strategy:get_device_strategy:45 - As num_cores_to_use is set to 1. using only ['/gpu:0']


## <font color="#000080" size="+3">5. Configuration</font>


In [10]:
# Model Benchmarking Parameters
batch_size = 8 # Number of samples processed per batch ex. 16
warm_up_batches = 2  # Number of initial batches to run to stabilize performance metrics ex. 10
num_batches = 10  # Total number of batches to process for benchmarking ex. 100
input_height = 64  # Height of the input images
input_width = 640  # Width of the input images
input_shape = (input_height, input_width, 3)  # Shape of the input data (H, W, C)

## <font color="#000080" size="+3">6. Keras cv attention models QPS checker</font>

In [11]:
from keras_cv_attention_models import efficientformer
from keras_cv_attention_models import efficientvit
# from keras_cv_attention_models import coatnet
# from keras_cv_attention_models import convnext
# from keras_cv_attention_models import davit
# from keras_cv_attention_models import fasternet
# from keras_cv_attention_models import eva02
# from keras_cv_attention_models import gcvit
# from keras_cv_attention_models import hiera
# from keras_cv_attention_models import maxvit
# from keras_cv_attention_models import mobilevit
# from keras_cv_attention_models import tinyvit
# from keras_cv_attention_models import volo

In [12]:
## Automatically find all model functions import it and test
import importlib
from tqdm import tqdm

from tf_tools.utils.function_finder_inspector import find_decorated_functions

package_name = 'keras_cv_attention_models'
decorator_name = 'register_model'
decorated_functions = find_decorated_functions(package_name, decorator_name)
print(f"\nTotal {len(decorated_functions)} functions found decorated with @{decorator_name}\n")

# for module_name, func_name, params in decorated_functions:
#     print(f"{module_name}.{func_name} with parameters: {params} \n")

Failed to import or process keras_cv_attention_models.keras_core_functional: No module named 'keras_core'
Failed to import or process keras_cv_attention_models.efficientnet.convert_effnetv2_model: No module named 'effnetv2_configs'
Failed to import or process keras_cv_attention_models.imagenet.tensorrt_engine: No module named 'tensorrt'

Total 437 functions found decorated with @register_model



In [13]:
# filter models list
decorated_functions_filtered = []

## optional
## names to keep -- to perform selected models benchmark only to save time
## Note: You can give partial name to select many matching names ex. Beit instead of BeitLargePatch16
name_filters = [
    "BeitLargePatch16", "BeitV2LargePatch16",
    "CAFormerB36", "ConvFormerB36",
    "CoAtNet3", "CoAtNet4",
    "ConvNeXtXlarge", "ConvNeXtV2Large",
    "CotNetSE152D",
    "DaViT_L", "DaViT_H",
    "DINOv2_ViT_Large14",
    "EfficientFormerL7", "EfficientFormerV2L",
    "EfficientNetV1L2", "EfficientNetV2XL",
    "EfficientViT_L3",
    "EvaGiantPatch14", 
    "EVA02BasePatch14", "EVA02LargePatch14",
    "FastViT_MA36",
    "FlexiViTLarge",
    "GCViT_Large",
    "GPViT_L4",
    "HieraHuge",
    "HorNetLargeGF",
    "IFormerLarge",
    "InceptionNeXtBase",
    "MaxViT_XLarge",
    "MetaTransformerLargePatch14",
    "MLPMixerH14",
    "MogaNetLarge",
    "ResMLP_B24",
    "ResNest269",
    "SwinTransformerV2Large",
    "TinyViT_21M",
    "UniformerLarge64",
    "VOLO_d5",
]

for module_name, func_name, params in tqdm(decorated_functions):
    if name_filters:
        if any(nf.lower().strip() in func_name.lower() for nf in name_filters):
            # keep only name matched models
            decorated_functions_filtered.append(
                (module_name, func_name, params) # append tuple
            )
        else:
            continue # skip if name not matching
    else:
        # if no any name filtering specified then remove only object detection models
        skip_models_patterns = ["YOLO", "EfficientDet"]
        if any(m.lower() in func_name.lower() for m in skip_models_patterns):
            continue # skip that
        else:
            decorated_functions_filtered.append(
                (module_name, func_name, params) # append tuple
            )
            
print(f"\n After filtering total {len(decorated_functions_filtered)} functions remaining\n")

100%|██████████| 437/437 [00:00<00:00, 65658.08it/s]


 After filtering total 40 functions remaining



In [14]:
#########################################

results_kecam = []

for module_name, func_name, params in tqdm(decorated_functions_filtered):
    try:  
        model_name = func_name
        print(f"Testing {model_name}")
        
        # Import the module and function
        module = importlib.import_module(module_name)
        function = getattr(module, func_name)

        # Parameters
        params = {
            "input_shape": input_shape,
            "num_classes": 1000,
            "pretrained": None, # we don't need pretrained weights here
        }

        # Call the function with dynamic parameters
        model = function(**params)

        ## Use the model as needed, e.g., print summary
        #print(model.summary())
        
        # get tps
        # max_batch_size = find_max_batch_size(model, input_size)
        qps, duration = benchmark_model(
            model, strategy, input_shape, 
            warm_up_batches=warm_up_batches, num_batches=num_batches, 
            batch_size=batch_size
        )
        print(f"qps:{qps}, duration: {duration}\n")
        results_kecam.append((model_name, batch_size, qps, duration))

    except Exception as e:
        print(f"Error in {module_name}.{func_name}.. {e}")
        
    finally:
        # clear the session
        tf_keras.backend.clear_session()

  0%|          | 0/40 [00:00<?, ?it/s]

Testing HorNetLargeGF
performing warmup...
Benchmarking...


  2%|▎         | 1/40 [00:16<10:45, 16.55s/it]

qps:13.36614717263459, duration: 5.985270023345947

Testing EfficientViT_L3
performing warmup...


I0000 00:00:1715757422.084866      24 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

qps:14.074373343176404, duration: 5.684089660644531

Testing DaViT_L
performing warmup...
Benchmarking...


 98%|█████████▊| 39/40 [10:10<00:15, 15.07s/it]

qps:14.26710444522113, duration: 5.607304573059082

Testing VOLO_d5
performing warmup...
Benchmarking...


100%|██████████| 40/40 [10:28<00:00, 15.72s/it]

qps:10.44864217822207, duration: 7.656497240066528



In [15]:
import pandas as pd

# Convert results to DataFrame
df_kecam = pd.DataFrame(results_kecam, columns=['Model', 'Max Batch Size', 'QPS', 'Time (s)'])

# sort records
df_kecam = df_kecam.sort_values(by="QPS", ascending=False)

# Display the results in a table
print(df_kecam)

# save result in file
df_kecam.to_csv('results_kecam.csv', index=False)  # Set index=False to avoid saving the index as a separate column
print("Saved result to file..")

                              Model  Max Batch Size        QPS   Time (s)
26                      TinyViT_21M               8  31.953607   2.503630
1                   EfficientViT_L3               8  31.390626   2.548532
9                  EVA02BasePatch14               8  26.220599   3.051036
35                   ConvNeXtXlarge               8  25.023536   3.196990
14                EfficientFormerL7               8  22.794743   3.509581
29                InceptionNeXtBase               8  22.187022   3.605711
30                         CoAtNet3               8  20.659648   3.872283
11      MetaTransformerLargePatch14               8  17.916251   4.465220
4                     FlexiViTLarge               8  17.896065   4.470257
15                       ResMLP_B24               8  17.726158   4.513104
5                DINOv2_ViT_Large14               8  17.085694   4.682280
22                    ConvFormerB36               8  16.499956   4.848498
7                BeitV2LargePatch16   

## <font color="#000080" size="+3">7. List all keras pretrained models and check QPS</font>


In [16]:
import numpy as np
import time
import tf_keras
from tf_keras.applications import *
from tf_keras.preprocessing import image
from tf_keras.utils import get_file
import tensorflow as tf
import pandas as pd
import inspect 
from tqdm import tqdm

# # List of models to test
# model_list = [MobileNetV2, Xception, ResNet50, VGG16, VGG19, InceptionV3]

# Get all model-creating functions without filtering
model_funcs = {
    name: obj for name, obj in inspect.getmembers(tf.keras.applications, inspect.isfunction)
}
model_list = []
for model_name, model_cls in model_funcs.items():
    model_list.append(model_cls)

print(f"Total models: {len(model_list)}")


Total models: 71


In [17]:
filtered_model_list = []

## optional
## names to keep -- to perform selected models benchmark only to save time
name_filters = [
    "VGG16", "VGG19",
    "ResNet50", "ResNet101",
    "ResNet50V2", "ResNet152V2",
    "InceptionResNetV2",
    "MobileNetV2",
    "DenseNet201",
    "NASNetLarge",
    "EfficientNetB2", "EfficientNetB7",
    "EfficientNetV2L",
    "ConvNeXtXLarge",
]

if name_filters:
    for model_class in model_list:
        model_name = model_class.__name__
        if any(nf.lower().strip() in model_name.lower() for nf in name_filters):
            filtered_model_list.append(model_class)
else:
    filtered_model_list = model_list # if no filtering
    
print(f"Total models after filtering: {len(filtered_model_list)}")

Total models after filtering: 15


In [18]:
# Main testing loop
results = []
for model_class in tqdm(filtered_model_list):
    model_name = model_class.__name__
    print("=" * 80)
    print(f"Testing {model_name}")
    
    try:
        with strategy.scope():
            model = model_class(
                weights=None, input_shape=input_shape
            )

        # max_batch_size = find_max_batch_size(model, input_size)
        qps, duration = benchmark_model(
            model, strategy, input_shape, 
            warm_up_batches=warm_up_batches, num_batches=num_batches, 
            batch_size=batch_size
        )
        print(f"qps:{qps}, duration: {duration}\n")
        results.append((model_name, batch_size, qps, duration))
        
    except Exception as e:
        print(f"Error in {model_name}.. {e}")
        
    finally:
        # clear the session
        tf_keras.backend.clear_session()
    
# # Print results
# for model_name, batch_size, qps, dur in results:
#     print(f"Model: {model_name}, Max Batch Size: {batch_size}, QPS: {qps:.2f}, Time: {dur:.2f}s")

  0%|          | 0/15 [00:00<?, ?it/s]

Testing ConvNeXtXLarge
performing warmup...
Benchmarking...


  7%|▋         | 1/15 [00:14<03:21, 14.37s/it]

qps:21.936058070380014, duration: 3.646963357925415

Testing DenseNet201
performing warmup...
Benchmarking...


 13%|█▎        | 2/15 [00:33<03:43, 17.19s/it]

...

qps:30.844178693541465, duration: 2.593682289123535

Testing VGG16
performing warmup...
Benchmarking...


 93%|█████████▎| 14/15 [03:01<00:08,  8.05s/it]

qps:166.56456689004716, duration: 0.48029422760009766

Testing VGG19
performing warmup...
Benchmarking...


100%|██████████| 15/15 [03:02<00:00, 12.20s/it]

qps:130.37986625686298, duration: 0.6135916709899902



In [19]:
# Convert results to DataFrame
df = pd.DataFrame(results, columns=['Model', 'Max Batch Size', 'QPS', 'Time (s)'])

# sort records
df = df.sort_values(by="QPS", ascending=False)

# Display the results in a table
print(df)

# save result in file
df.to_csv('results_tf_keras.csv', index=False)  # Set index=False to avoid saving the index as a separate column
print("Saved result to file..")

              Model  Max Batch Size         QPS   Time (s)
12            VGG16               8  166.564567   0.480294
13            VGG19               8  130.379866   0.613592
5       MobileNetV2               8   30.983825   2.581992
11       ResNet50V2               8   30.844179   2.593682
10         ResNet50               8   29.033461   2.755441
0    ConvNeXtXLarge               8   21.936058   3.646963
2    EfficientNetB2               8   16.548078   4.834398
8       ResNet101V2               8   16.035759   4.988850
7         ResNet101               8   14.920223   5.361850
9       ResNet152V2               8   10.574416   7.565430
1       DenseNet201               8    8.395875   9.528489
3    EfficientNetB7               8    6.875056  11.636269
4   EfficientNetV2L               8    5.551857  14.409594
6       NASNetLarge               8    5.371612  14.893109
Saved result to file..
